In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

In [2]:
#chargements des caractéristiques
carac = pd.read_csv('carcteristiques-2022.csv', sep=';')
#conversion en chaine
cols_to_cast = ['Accident_Id', 'lum', 'agg', 'int', 'atm', 'col']
carac[cols_to_cast] = carac[cols_to_cast].astype('int')
#conversion des latitudes/longitudes
carac['lat']= carac['lat'].str.replace(',','.').astype('float')
carac['long']= carac['long'].str.replace(',','.').astype('float')

# suppression de l'adresse qui ne sert à rien
carac = carac.drop('adr', axis = 1)

#on ajoute un créneau horaire
carac['hh'] = carac['hrmn'].str[:2]

# traitement des -1 en NA
carac.replace({-1:np.nan}, inplace=True)

# gestion des NAs
# lum : 2 nas => valuers la plus fréquentes (1)
carac.loc[carac['lum'].isna(),'lum'] = 1

# int : 7 nas => valuers la plus fréquentes (1)
carac.loc[carac['int'].isna(),'int'] = 1

# atm : 1 nas => valuers la plus fréquentes (1)
carac.loc[carac['atm'].isna(),'atm'] = 1

# col : 63 nas => valuers la plus fréquentes (6) : atre collision
carac.loc[carac['col'].isna(),'col'] = 6

In [3]:
# chargement des usagers
usagers = pd.read_csv('usagers-2022.csv', sep=';')

#suppression du num_veh qui est en doublon avec id_vehicule
usagers.drop('num_veh', axis = 1, inplace = True)

#correction dees valeurs inconnue actp
usagers.loc[usagers['actp'].isin([' -1','8','9']),'actp'] = -1

# place manquante : tous les usagers concernés sont conduxcteurs : donc on remplace par 1
usagers.loc[usagers['place']== -1 ,'place'] = 1

# catu : on remplca e0 par 1 = le plus nombreux
#usagers.loc[usagers['catu']== 0 ,'catu'] = 1
usagers['catu'] = usagers['catu'].replace(0,1)


# gravité : bcp d'information manquantes pour ces usagers : on supprime
usagers.dropna(subset=['grav'], inplace=True)

#sexe : le plus courant = H
usagers['sexe'] = usagers['sexe'].replace({0:-1})
usagers.loc[usagers['sexe']== -1 ,'sexe'] = 1

# trajet :  catégorie 0 : non renseignée conservée car très nombreuse
usagers.loc[usagers['trajet']== -1, 'trajet'] = 0



#secu  : un champ binaire par équipement
for i in range(1,10) :
    usagers['secu_'+str(i)] = ((usagers['secu1']==i) | (usagers['secu2']==i) | (usagers['secu3']==i))*1

usagers = usagers.drop(['secu1','secu2','secu3'], axis = 1)



#année de naissance : année moyenne
usagers.loc[usagers['an_nais'] == 0,'an_nais'] = 1987
usagers.loc[usagers['an_nais'].isna(),'an_nais'] = 1987

#sexe : 1 est le plus nombreux
usagers.loc[usagers['sexe'] == 0,'sexe'] = 1

# locp. pour les usagers non piétons, la valeur -1 est remplacé"e par 0 sans objet
# pour les piétonzs, on met 9 : inconnue
usagers.loc[usagers['catu']!=3,'locp'] = 0
usagers.loc[(usagers['catu']==3) & (usagers['locp']==-1),'locp'] = 9

#actp. pour les non piétons, valeurmanquantes = 0 pour sans objet
# pour les piétons : B = inconnu
usagers.loc[usagers['catu']!=3,'actp'] = 0
usagers.loc[(usagers['catu']==3) & (usagers['actp'] == -1),'actp'] = 'B'

#on recode actp A et B en 10 ,11
usagers.loc[usagers['actp']=='A', 'actp'] = '10'
usagers.loc[usagers['actp']=='B', 'actp'] = '11'

#etatp : pour les non piétons, on mets à 0 = sans objet
#pour le smiétons, 1 (le plus fréquent)
usagers.loc[(usagers['catu']!=3)&(usagers['etatp']==-1), 'etatp'] = 0
usagers.loc[(usagers['catu']==3)&(usagers['etatp']==-1), 'etatp'] = 1

#cast
usagers[['an_nais','actp']] = usagers[['an_nais','actp']].astype('int')


In [74]:
# chargements ds véhicules
vehicules = pd.read_csv('vehicules-2022.csv', sep=';')
#suppression du code redondant
vehicules = vehicules.drop('num_veh', axis = 1)

#catv : le plus fréquent = 7
vehicules['catv'] = vehicules['catv'].replace({-1:7})

# obs : le plus fréquent = 0
vehicules['obs'] = vehicules['obs'].replace({-1:0})


In [79]:
vehicules['obsm'].value_counts()

obsm
 2    66333
 0    17638
 1     8551
 9     1621
 6      119
 4      102
 5       81
-1       48
Name: count, dtype: int64

In [61]:
# chargement des lieux
lieux = pd.read_csv('lieux-2022.csv', sep=';')
# suppression du nom de voie
lieux = lieux.drop(['voie','v1','v2'], axis = 1)
# conversion en chaine
#cols_to_cast = ['Num_Acc', 'catr', 'circ', 'vosp', 'prof', 'plan', 'surf', 'infra', 'situ']
#lieux[cols_to_cast] = lieux[cols_to_cast].astype('str')
#convserion de la largeur de la chaussée :
lieux['larrout'] = lieux['larrout'].str.replace(',','.').astype('float')

#nbr de voies:
lieux.loc[lieux['nbv']=='#ERREUR', 'nbv'] = '-1'
lieux['nbv'] = lieux['nbv'].astype('int')
lieux['pr'] = lieux['pr'].replace('(1)','1')
lieux['pr'] = lieux['pr'].astype('int')
lieux['pr1'] = lieux['pr1'].replace('(1)','1')
lieux['pr1'] = lieux['pr1'].astype('int')

#circ : régilme le plus fréquent pour le mê type de route, et pour les autres ! sans objet (parking et voies privées)
for key, val in lieux[['catr','circ']].groupby('catr')['circ'].agg(pd.Series.mode).to_dict().items() :
    lieux.loc[( lieux['circ'] == -1 ) & (lieux['catr']== key), 'circ'] = val

lieux['circ'] = lieux['circ'].replace({-1:0})

#nbv : idem circ
for key, val in lieux[['catr','nbv']].groupby('catr')['nbv'].agg(pd.Series.mode).to_dict().items() :
    lieux.loc[( lieux['nbv'] == -1 ) & (lieux['catr']== key), 'nbv'] = val

lieux['nbv'] = lieux['nbv'].replace({-1:0})

#vosp -> 0 le plus fréquent



c:\Temp\eric.favre\temp\10\ipykernel_5324\3562934176.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  lieux = pd.read_csv('lieux-2022.csv', sep=';')


In [64]:
lieux.loc[lieux['vosp']==-1,['catr','vosp']].value_counts()

catr  vosp
3     -1      74
4     -1      40
2     -1      23
1     -1      19
7     -1       7
6     -1       1
Name: count, dtype: int64

In [69]:
lieux[['catr','vosp']].groupby('catr')['vosp'].agg(pd.Series.mode)

catr
1    0
2    0
3    0
4    0
5    0
6    0
7    0
9    0
Name: vosp, dtype: int64

,Num_Acc,catr,circ,nbv,vosp,prof,pr,pr1,plan,lartpc,larrout,surf,infra,situ,vma
242,202200000243,2,2,4,-1,1,22,70,1,NaN,-1.0,1,0,8,-1
483,202200000484,3,1,2,-1,1,1,1,1,NaN,-1.0,2,5,1,-1
545,202200000546,7,2,2,-1,1,1,1,1,NaN,7.0,1,0,1,-1
566,202200000567,4,2,2,-1,1,1,1,1,NaN,-1.0,1,0,1,-1
597,202200000598,3,2,2,-1,2,22,335,2,NaN,-1.0,1,0,1,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53021,202200053022,3,2,2,-1,1,1,1,1,NaN,-1.0,1,0,1,-1
53530,202200053531,1,1,4,-1,1,1,1,3,NaN,-1.0,1,3,1,50
54008,202200054009,3,3,2,-1,1,32,50,1,NaN,4.0,2,5,1,50
54147,202200054148,2,2,2,-1,1,64,500,1,NaN,-1.0,1,5,1,-1
